In [1]:
import sys
sys.path.append("/home/ubuntu/anaconda2/lib/python2.7/site-packages")


In [2]:
import gensim

In [3]:
import os

In [5]:
import nltk
import os
import traceback
from sklearn.externals import joblib
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import numpy as np
import json
from nltk.tokenize import sent_tokenize
from concurrent import futures
from shutil import copyfile
import senticnet
import re

In [9]:
tokenizer = RegexpTokenizer('\w+')
stemmer = PorterStemmer()

In [10]:
def preprocess(x):
    return x.replace('\n', ' ').replace('\r', '').replace('\x0C', '').lower()

In [11]:
class doc():
    def __init__(self, title, company, concepts, stemmed_concepts, cashtag, source):
        if isinstance(title, float):
            title = ''
        self.content = title
        self.company = company
        self.source = source
        self.cashtag = cashtag

        if isinstance(concepts, str):
            self.concepts = concepts
        else:
            self.concepts = ''

        # if isinstance(stemmed_concepts, str):
        #     self.concepts = stemmed_concepts
        # else:
        #     self.concepts = ''


In [12]:
import csv

In [13]:
with open("Microblog_Trainingdata.json") as file:
    trainingdata = json.load(file)

In [14]:
outputTrainingFile = (open('Microblog_Trainingdata.csv','wb+'))
outputtraining = csv.writer(outputTrainingFile)
outputtraining.writerow(trainingdata[0].keys())  # header row
for row in trainingdata:
    outputtraining.writerow(row.values())

In [30]:
with open("Microblog_Trialdata.json") as file:
    trialdata = json.load(file)

In [31]:
outputTrialFile = (open('Microblog_Trialdata.csv','wb+'))
outputTrial = csv.writer(outputTrialFile)
outputTrial.writerow(trialdata[0].keys())  # header row
for row in trialdata:
    outputTrial.writerow(row.values())

In [23]:
with open("Microblogs_Testdata_withscores.json") as file:
    testdata = json.load(file)

In [24]:
outputTestFile = (open('Microblogs_Testdata.csv','wb+'))
outputTest = csv.writer(outputTestFile)
outputTest.writerow(testdata[0].keys())  # header row


In [20]:
# Write CSV Header, If you dont need that, remove this line
#f.writerow(["source", "cashtag", "sentiment score", "id", "spans"])

In [26]:
import pandas as pd

In [27]:
df_training = pd.read_csv('Microblog_Trainingdata.csv')


In [28]:
df_training.isnull().sum()

source             0
spans              0
cashtag            1
id                 1
sentiment score    1
dtype: int64

In [32]:
df_trial=pd.read_csv('Microblog_Trialdata.csv')


In [33]:
df_test=pd.read_csv('Microblogs_Testdata_withscores.csv')

In [34]:
df_test.shape[0]

794

In [35]:
df_training_trial=df_training.append(df_trial)

In [36]:
df_training_trial.head()

,cashtag,id,sentiment score,source,spans
0,$FB,7.196594e+17,0.366,twitter,[u'watching for bounce tomorrow']
1,$LUV,7.199043e+17,0.638,twitter,[u'record number of passengers served in 2015']
2,$NFLX,5.329774e+06,-0.494,stocktwits,[u'out $NFLX -.35']
3,$DIA,7.198915e+17,0.460,twitter,"[u'Looking for a strong bounce', u'Lunchtime r..."
4,$PLUG,2.009125e+07,0.403,stocktwits,[u'Very intrigued with the technology and grow...


In [37]:
df_training_trial_test=df_training_trial.append(df_test)

In [38]:
df_training_trial_test.head()

,cashtag,id,sentiment score,source,spans,text
0,$FB,7.196594e+17,0.366,twitter,[u'watching for bounce tomorrow'],NaN
1,$LUV,7.199043e+17,0.638,twitter,[u'record number of passengers served in 2015'],NaN
2,$NFLX,5.329774e+06,-0.494,stocktwits,[u'out $NFLX -.35'],NaN
3,$DIA,7.198915e+17,0.460,twitter,"[u'Looking for a strong bounce', u'Lunchtime r...",NaN
4,$PLUG,2.009125e+07,0.403,stocktwits,[u'Very intrigued with the technology and grow...,NaN


In [65]:
df_training_trial_test.to_csv('mb_train_trial_test_new_raw.csv')

In [40]:
df_training_trial_test

,cashtag,id,sentiment score,source,spans,text
0,$FB,7.196594e+17,0.366,twitter,[u'watching for bounce tomorrow'],NaN
1,$LUV,7.199043e+17,0.638,twitter,[u'record number of passengers served in 2015'],NaN
2,$NFLX,5.329774e+06,-0.494,stocktwits,[u'out $NFLX -.35'],NaN
3,$DIA,7.198915e+17,0.460,twitter,"[u'Looking for a strong bounce', u'Lunchtime r...",NaN
4,$PLUG,2.009125e+07,0.403,stocktwits,[u'Very intrigued with the technology and grow...,NaN
5,$GMCR,5.819749e+06,0.000,stocktwits,"[u'short worked', u'puts up']",NaN
6,$IBM,7.097412e+17,-0.296,twitter,[u'overbought'],NaN
7,$JOSB,1.789297e+07,-0.546,stocktwits,"[u'absolute garbage still up', u'stores TOTALL...",NaN
8,$CSTM,7.098343e+17,-0.438,twitter,[u'Biggest Market Losers'],NaN
9,$PYPL,7.084814e+17,0.408,twitter,[u'Love this company long time.'],NaN


In [41]:
df_training_trial_test.isnull().sum()

cashtag               1
id                    1
sentiment score       1
source                0
spans                10
text               1693
dtype: int64

In [42]:
#Preprocessing
def replace_url(content):
    return re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'URL', content)

In [43]:
#TOKENIZE DATA with/out Named Entity and numbers Removal
def remove_named_entities( content ):
    if isinstance(content, float):
        return ''

    sentences = nltk.sent_tokenize(content)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

    def extract_entity_names(t):
        entity_names = []

        if hasattr(t, 'label') and t.label:
            if t.label() == 'NE':
                entity_names.append(' '.join([child[0] for child in t]))
            else:
                for child in t:
                    entity_names.extend(extract_entity_names(child))

        return entity_names

    entity_names = []
    for tree in chunked_sentences:
        entity_names.extend(extract_entity_names(tree))
    entity_names = list(reversed(sorted(entity_names, key=len)))

    for ne in entity_names:
        content = content.replace(ne, 'NAMEDENTITY')

    return content

In [44]:
def replace_numbers(tokens):
    for i in range(len(tokens)):
        if tokens[i].isdigit():
            tokens[i] = 'CC'

    return tokens

In [45]:
def tokenize_csv_file( source_file_path, dest_file_path, column_names, should_remove_NE, should_remove_numbers ):
    """
    Tokenizes contents of all the columns mentioned in the params
    :param source_dir: (str) Directory of reviews
    :return:
    """
    source_df = pd.read_csv(source_file_path)

    for index, row in source_df.iterrows():

        for target_column in column_names:
            target_text = row[target_column]
            target_text = replace_url(target_text)
            if should_remove_NE:
                ne_removed_content = remove_named_entities(target_text)
            else:
                ne_removed_content = target_text

            tokens = tokenizer.tokenize(ne_removed_content)
            if should_remove_numbers:
                tokens = replace_numbers(tokens)
            source_df.set_value(index, target_column, ' '.join(tokens))
    source_df.to_csv(dest_file_path, index=False)

In [46]:
column_names=['spans']

In [50]:
for index, row in df_training_trial_test.iterrows():
    for target_column in column_names:
            target_text = row[target_column]
            #target_text = replace_url(target_text)
            ne_removed_content = remove_named_entities(target_text)
            #ne_removed_content = target_text
            tokens = tokenizer.tokenize(ne_removed_content)
            tokens = replace_numbers(tokens)
            df_training_trial_test.set_value(index, target_column, ' '.join(tokens))
    df_training_trial_test.to_csv('mb_train_trial_test_new_prs.csv', index=False)

In [49]:
#with open('names.csv') as namescsv:
 #   namereader = csv.reader(namescsv)
for row in df_training_trial_test:
    for cell in row:
        cell = re.sub(r'[^\w=]', '',cell)


In [64]:
from senticnet.senticnet import Senticnet
def extract_sentic_concepts_and_scores_csv():
    #source_dir = 'mb_train_trial_test_new_prs.csv'
    source_df = pd.read_csv('mb_train_trial_test_new_prs.csv')
    source_df['concepts'] = ''
    source_df['polarity'] = ''
    source_df['attention'] = ''
    source_df['pleasantness'] = ''
    source_df['aptitude'] = ''
    source_df['sensitivity'] = ''
    sn = Senticnet()

    counter = 0

    for index, row in source_df.iterrows():
        concepts = []
        concept_scores = {'polarity' : 0, 'attention': 0, 'pleasantness': 0, 'aptitude': 0, 'sensitivity': 0}
        stemmed_concepts = []
        stemmed_concept_scores = {'polarity': 0, 'attention': 0, 'pleasantness': 0, 'aptitude': 0, 'sensitivity': 0}

        if isinstance(row['spans'], float):
            content = ''
            stemmed_title = ''
        else:
            content = '_' + row['spans'].replace(' ', '_').lower() + '_'
            stemmed_title = '_' + '_'.join([stemmer.stem(t) for t in row['spans'].lower().split()]) + '_'
        # print(stemmed_title)
        for concept_key in sn.data.keys():
            if '_' + concept_key +'_' in content:
                concepts.append(concept_key)
                concept_data = sn.concept(concept_key)
                concept_scores['polarity'] += float(concept_data['polarity_intense'])
                concept_scores['attention'] += float(concept_data['sentics']['attention'])
                concept_scores['pleasantness'] += float(concept_data['sentics']['pleasantness'])
                concept_scores['aptitude'] += float(concept_data['sentics']['aptitude'])
                concept_scores['sensitivity'] += float(concept_data['sentics']['sensitivity'])
            if '_' + concept_key +'_' in stemmed_title:
                stemmed_concepts.append(concept_key)
                concept_data = sn.concept(concept_key)
                stemmed_concept_scores['polarity'] += float(concept_data['polarity_intense'])
                stemmed_concept_scores['attention'] += float(concept_data['sentics']['attention'])
                stemmed_concept_scores['pleasantness'] +=float(concept_data['sentics']['pleasantness'])
                stemmed_concept_scores['aptitude'] +=float(concept_data['sentics']['aptitude'])
                stemmed_concept_scores['sensitivity'] +=float(concept_data['sentics']['sensitivity'])

        # print(concept_scores)

        if len(concepts) > 0:
            for k, v in concept_scores.items():
                concept_scores[k] /= len(concepts)
        else:
            print('>> 0  >>', row['spans'])
        if len(stemmed_concepts) > 0:
            for k, v in stemmed_concept_scores.items():
                stemmed_concept_scores[k] /= len(stemmed_concepts)
            else:
                print('>> 1  >>', stemmed_title)

        # print(concept_scores)
        source_df.set_value(index, 'polarity', concept_scores['polarity'])
        source_df.set_value(index, 'attention', concept_scores['attention'])
        source_df.set_value(index, 'pleasantness', concept_scores['pleasantness'])
        source_df.set_value(index, 'aptitude', concept_scores['aptitude'])
        source_df.set_value(index, 'sensitivity', concept_scores['sensitivity'])
        source_df.set_value(index, 'concepts', ' '.join(concepts))
        source_df.set_value(index, 'stemmed_polarity', stemmed_concept_scores['polarity'])
        source_df.set_value(index, 'stemmed_attention', stemmed_concept_scores['attention'])
        source_df.set_value(index, 'stemmed_pleasantness', stemmed_concept_scores['pleasantness'])
        source_df.set_value(index, 'stemmed_aptitude', stemmed_concept_scores['aptitude'])
        source_df.set_value(index, 'stemmed_sensitivity', stemmed_concept_scores['sensitivity'])
        source_df.set_value(index, 'stemmed_concepts', ' '.join(stemmed_concepts))
            

        print(counter)
        counter += 1
        # break
    source_df.to_csv('mb_train_trial_test_new_prs1.csv', index=False)
    


In [52]:
extract_sentic_concepts_and_scores_csv()

('>> 1  >>', u'_consum_keep_cautiou_stanc_')
0
1
('>> 1  >>', u'_everi_reason_to_be_namedent_')
2
('>> 1  >>', u'_namedent_need_anoth_cc_to_all_time_high_')
3
('>> 1  >>', u'_long_setup_')
4
('>> 1  >>', u'_will_be_a_solid_long_term_investmen_passiv_also_ad_namedent_today_incom_')
5
6
('>> 1  >>', u'_namedent_win_new_starwood_deal_by_rais_cash_portion_of_bid_')
7
('>> 0  >>', 'NAMEDENTITY')
8
('>> 0  >>', 'NAMEDENTITY AppleStores are NAMEDENTITY in NAMEDENTITY MS must be preparing to liquidate NAMEDENTITY')
('>> 1  >>', u'_namedent_applestor_are_namedent_in_namedent_ms_must_be_prepar_to_liquid_namedent_')
9
('>> 1  >>', u'_namedent_star_analyst_namedent_from_namedent_reiter_a_buy_')
10
('>> 1  >>', u'_excit_big_valu_hope_')
11
('>> 0  >>', 'NAMEDENTITY')
12
('>> 0  >>', 'NAMEDENTITY')
13
('>> 1  >>', u'_namedent_s_namedent_t_go_anywher_')
14
('>> 0  >>', 'NAMEDENTITY NAMEDENTITY')
15
('>> 0  >>', 'not guaranteed')
16
17
('>> 0  >>', 'longed')
('>> 1  >>', u'_long_')
18
('>> 1  >>', u'_